# Week 3-- Dynamic programming, alignment, sequences
In this week we will turn our attention to a slightly more meaty computational topic-- sequence alignment. Our goal through the end of week three is to create a multiple alignment from a set of SARS-CoV-2 genomes that we might play with in coming weeks.

As we talked about in lecture, the problem of sequence alignment arises because of the need for us to be comparing apples to apples between genomes. That is to say that we want to try to ensure that we are considering *homologous* traits, in this case basepairs of the genome, when we start to consider the evolutionary relationships among DNA sequences. 

Consider the number of ways in which two DNA sequences can be aligned one to another. It's easy enough to imagine that as these sequences themselves get longer, so too does the number of possible alignments. To figure out which alignment among all of these possibilities is the best will be a challenge, so we're gonna need a clever algorithm that allows us to compute stuff cheaply.

Our solution comes from family of algorithms called Dynamic Programming. Dynamic programming exploits the fact that many problems can actually be decomposed into smaller subproblems, that when combined can solve a larger whole problem. This substructure solution comes from what we call _recursion_.


## The Fibonacci sequence
The Fibonacci sequence was famously introduced by a 13th Century Italian mathematician of the same name to describe the growth of rabbit populations. The sequence of Fibonacci numbers goes like this:

$F_0 = 0, F_1 = 1$ and $F_n = F_{n-1} + F_{n-2}$

**Exercise 1:**
In class, write a function to compute all of the Fibonacci series up to some defined $i$ th term. Your function should take $i$, the last number in the series to calculate as input and return $F_i$

## Recursive functions
In mathematics recursive equations are those which are defined on the basis of previous terms of the same equation. For instance the Fibonacci series above is _recursive_ in that $F_n$ is calculated from $F_{n-1}$ and $F_{n-2}$. In computer programming the analog is a _recursive function_ that calls itself to achieve the desired output.

As a very simple example check out the recursive function below `recFunc()`

In [ ]:
def recFunc(anInt):
    print(anInt)
    if anInt > 0:
        anInt -= 1
        recFunc(anInt)
        
recFunc(3)

here `recFunc()` is making calls to `recFunc()` from within it's own code until something happens! Can you follow the logic here? If you can the next thing to do is switch the order of the `if` block and the `print` statement. Now what's happening? 

In [ ]:
 def recFunc(anInt):
    if anInt > 0:
        anInt -= 1
        recFunc(anInt)
    print(anInt)
        
recFunc(3)

**Exercise 2:** In class, write a new Fibonacci function that outputs the $i$ th term, but this time do it using a recursive function.

## Dynamic Programming version
So let's say that we were writing a program that had to pull out lots of Fibonacci numbers over and over again, what would be a good way to do this efficient? Well what if we fill in a table to do this for us?

**Exercise 3:** Write a new Fibonacci function, but this one will take as input $i$ the last term to be computed but will return a numpy array of Fibonacci numbers up to and including $F_i$. Can you think of a clever way to do this that uses the structure of the problem?



## Efficiency comparison
Let's see how long a naive implementation of the function which returns the list of Fibonacci numbers is versus a Dynamic programming implementation. To do this I'll use a jupyter built in trick called `%timeit` which will return the amount of time used by the computer to evaluate a block of code

In [ ]:
import numpy as np

def F(anInt):
    if anInt == 0 or anInt == 1:
        return 1
    else:
        return F(anInt - 1) + F(anInt - 2)

In [ ]:
%%timeit
n = 20
fs = np.zeros(n)
for i in range(n):
    fs[i] = F(i)
fs

In [ ]:
def F_dp(aInt):
    res = np.ones(aInt)
    if aInt > 1:
        for i in range(2,aInt):
            res[i] = res[i-1] + res[i-2]
    return(res)

In [ ]:
%%timeit
F_dp(20)

## Tower of Hanoi Game
Okay let's look at a slightly more difficult dynamic programming problem-- solving for the minimum number of moves necessary to win the Tower of Hanoi game. Recall that the game consist of three rods (call them A, B, and C), onto which $n$ disks of increasing size sit. At the start of the game all $n$ disks are on the A rod. The objective is to move the $n$ disks to another rod (B or C). The rules are that 
1. Only 1 disk can be moved at a time
2. Each move consists of moving the upper disk on a rod to the top of the pile on a different rod
3. No larger disk may be placed on a smaller disk. 

Here's what an animation of the game looks like start from 6 disks

<img src=imgs/Tower_of_Hanoi.gif>

What's cool about this, seemingly hard problem is that we can break it up into the optimal move to do from any position, using a simple recursion. Here's what it looks like. Let's spend a bit of time analyzing it.

In [ ]:
def TowerOfHanoi(n , from_rod, to_rod, aux_rod): 
    if n == 1: 
        print(f"Move disk 1 from rod {from_rod} to rod {to_rod}") 
        return
    TowerOfHanoi(n-1, from_rod, aux_rod, to_rod) 
    print(f"Move disk {n} from rod {from_rod} to rod {to_rod}")
    TowerOfHanoi(n-1, aux_rod, to_rod, from_rod) 
    
TowerOfHanoi(3 , "A", "C", "B")

**Exercise:** change the starting value of n (= the number of disks) to 2. Does this make sense? Now try a larger number

## Sequence Alignment
Now that we've spent a bit of time learning the context for dynamic programming, let's dig in to a more germane (and complex!) model-- that of global alignment of two sequences. The alignment method we will look at, the so-called Needleman-Wunsch algorithm, was developed by Saul B. Needleman and Christian D. Wunsch and published in 1970 in a [landmark publication](https://www.sciencedirect.com/science/article/abs/pii/0022283670900574?via%3Dihub). A key innovation here was that they represent a sequence alignment as a matrix, and fill in scores within that matrix using a dynamic programming approach, essentially breaking the large problem (global alignment) into a repeated series of smaller problems (should the next state be a gap, a match, or a mismatch). 

As we spent time thinking about this in class I won't belabor my description too much 


In [ ]:
import numpy as np

# set sequences
seq_1 = "TAGACTA"
seq_2 = "TACGGACGG"

# define penalties for alignment
match = 4
m_match = -3
gap_penalty = -2

subst_matrix = {
'A': {'A': match,'C':m_match,'G':m_match,'T':m_match}, 
'C': {'A':m_match,'C': match,'G':m_match,'T':m_match}, 
'G': {'A':m_match,'C':m_match,'G': match,'T':m_match},
'T': {'A':m_match,'C':m_match,'G':m_match,'T': match},
}


# this will store our dynamic programming matrix
dp_matrix = np.ndarray(shape=(len(seq_1)+1,len(seq_2)+1), dtype=int)
dp_matrix.fill(0)

# storage for trace back matrix
# 0 for up, 1 for diag, 2 for left
back_ptr = np.ndarray(shape=(len(seq_1)+1,len(seq_2)+1), dtype=int)
back_ptr.fill(-9)

# fill the DP matrix
for i in range(len(seq_1)+1):
    for j in range(len(seq_2)+1):
        if i==0 and j==0: continue # skip the first cell

        char_1 = seq_1[i-1] # current character at seq 1
        char_2 = seq_2[j-1] # current character at seq 2

        scores = np.array([-999,-999,-999])
        if( i>0 and j>0 ):  
            # score diagonal
            scores[1] = subst_matrix[char_1][char_2] + dp_matrix[i-1][j-1]
            pass
        if( i>0 ): 
            # score up: gap in sequence 2
            scores[2] = gap_penalty + dp_matrix[i-1][j]
            pass
        if( j>0 ): 
            # score left: gap in sequence 1
            scores[0] = gap_penalty + dp_matrix[i][j-1]
            pass

        # select the best previous cell
        best = np.max(scores)
        dp_matrix[i,j]=best
        # record which move it was, choices
        # are 0->left, 1->diagonal, 2->up
        for k in range(3):
            if scores[k] == best:
                back_ptr[i,j] = k

print("Dynamic programming matrix:")
print(dp_matrix)
print("\n traceback pointers:")
print(back_ptr)

# read out the backtrace to get the best alignment
aln_1 = ""
aln_2 = ""
i=len(seq_1)
j=len(seq_2)

while i>0 or j>0:
    if back_ptr[i,j] == 0: # left
        aln_1 += "-"
        aln_2 += seq_2[j-1]
        j -= 1
    if back_ptr[i,j] == 1: # diag
        aln_1 += seq_1[i-1]
        aln_2 += seq_2[j-1]
        i -= 1
        j -= 1
    if back_ptr[i,j] == 2: # up
        aln_1 += seq_1[i-1]
        aln_2 += "-"
        i -= 1

aln_1 = aln_1[::-1] # reverses the string
aln_2 = aln_2[::-1]

print("\nAlignment:")
print(aln_1)
print(aln_2)



**Exercise:** change the `gap_penalty` parameter above to equal 0. What is the reported alignment that you get out? Next return the `gap_penalty` to -2 but increase the mismatch penalty. Now what do you see?

the Needleman-Wunsch algorithm above performs _global_ alignment, i.e. it seeks the best alignment of all the bases in the two sequences. If we want to instead find the best alignment of subsequences between the two sequences we can use the related [Smith-Waterman algorithm](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm). Recall from lecture that the only real differences here are that in Smith-Waterman scores in our DP matrix can't go below zero, and that the traceback starts at the *max* value cell and goes back until it encounters the first zero. 

# Biopython 
Now that we know something about sequence alignment we are ready to move on to our case study for the week-- aligning SARS-CoV-2 genome data. To do this we will rely on the python package `biopython`.

For anyone working with DNA or protein sequence information, `biopython` provides an extremely helpful set of tools. biopython gives the user the ability to programatically interact with biological sequence data and includes plugins to popular alignment and homology search algorithms such as BLAST or CLUSTAL, phylogenetic packages, and much more. We will barely scratch the surface in the kinds of things that one can accomplish, so if you are interested you can start reading the documentation [here](https://biopython.org/wiki/Documentation).


## Working with sequences
The first use case for us will be working with DNA sequences using biopython. biopython provides for us a `Seq` object, that contains at it's heart a string of biological sequence but that "knows" how to do certain tricks

In [ ]:
from Bio.Seq import Seq
my_seq = Seq("AGTATCTTTGGT")
print(my_seq)

print(my_seq.complement())
print(my_seq.reverse_complement())

sequences generally behave as strings, meaning that you can index them and iterate over them, etc.

In [ ]:
for c in my_seq:
    print(c)
    
print("here is my_seq[0]: ",my_seq[0])

these sequences know a lot of different tricks of course. for instance if we have gaps inserted into a sequence we can remove them

In [ ]:
my_seq = Seq("AGTAT----CTTTGG----T")
print(my_seq) # original seq
print(my_seq.ungap()) # ungapped version

We can ostuff like like transcribe and translate DNA. Let's program the central dogma real quick

In [ ]:
dna = Seq("ATGAAAGGAGGGAGGTAG")
mrna = dna.transcribe()
prot = mrna.translate()
print("DNA: ",dna)
print("mRNA: ",mrna)
print("protein: ",prot)


there are many other basic sequence utilities that biopython provides. you have to wade through the sequtils documentation a bit to find out everything that it can do out of the box.

## Reading in sequences
Perhaps the single most useful thing that biopython provides is basic utilities to read and write from common data formats such as fasta and fastq. These parsers really aid in our ability to quickly make headway on even sophisticated datasets. We will work with a set of SARS-CoV-2 genomes that I have provided in a data directory



In [ ]:
from Bio import SeqIO
for seq_record in SeqIO.parse("data/gisaid_cov2020_sequences_15mar2020.fasta", "fasta"):
    print("id: ",seq_record.id,"length: ",len(seq_record.seq))


In [ ]:
# read in large number of sequences as list of SeqRecords
covid_records = list(SeqIO.parse("data/gisaid_cov2020_sequences_15mar2020.fasta", "fasta"))
# write first 25 into small file
SeqIO.write(covid_records[0:25], "data/covid_small.fasta", "fasta")
# read small file
covid_records = list(SeqIO.parse("data/covid_small.fasta", "fasta"))
print(len(covid_records))

`SeqIO.parse()` actually returns a specific python object called an iterator that can be operated on in a different way, asking it to give you the next object over and over

In [ ]:
#a slightly different way to deal with the iterator that SeqIO.parse returns    
record_iterator = SeqIO.parse("data/covid_small.fasta", "fasta")
first_record = next(record_iterator)
print(first_record.id)
print(first_record.description)

second_record = next(record_iterator)
print(second_record.id)
print(second_record.description)

### Getting sequences straight from Genbank
biopython has code that allows us to download sequences directly from genbank and suck them up in to memory for doing stuff. this is particularly useful for snatching _reference sequences_, the an official representation of a sequence that is often well annotated. We will use this interface to grab a reference genome for SARS-CoV-2

In [ ]:
from Bio import Entrez
from Bio import SeqIO
# need this fake email for the service to work
Entrez.email = "fake_email@thanks.com"
with Entrez.efetch(db="nucleotide", rettype="fasta", retmode="text", id="NC_045512.2") as handle:
    seq_record = SeqIO.read(handle, "fasta")
print(seq_record.id, seq_record.description)

## SeqRecords
the workhorse here is a class called a `SeqRecord` that biopython provides to us. we can get a bunch of information here just be asking the object to print itself. these are smart. full documentation on these [here](https://biopython.org/wiki/SeqRecord)

In [ ]:
print(seq_record)

we can get a bit more information if we retreive files in the so-called genbank format, in particular we will get annotation features associated with the sequence, if they exist in the genbank entry. The genbank page for this entry can be found [here](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2), we can look it over in class together to see all the neat information that they include

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "A.N.Other@example.com"
with Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="NC_045512.2") as handle:
    seq_record = SeqIO.read(handle, "gb")
print(seq_record.id, seq_record.description)
#gene_list = [ x  for x in seq_record.features if x.type == 'CDS']

perhaps the biggest thing we get with the genbank file format is _annotations_, that is features of the genome that are represented and stored as so-called SeqFeatures. I can get these by looking at the `.features` attribute associated with my sequence records.

Let's look at the first four features entry from our file

In [ ]:
for i,x in enumerate(seq_record.features):
    if i < 4:
        print(x)


## What genes are in here? Grabbing feature sequences
okay lets get a list of gene names from our features. One of the more interesting loci in the SARS-CoV-2 genome is the 'S' locus or Spike protein. The New York Times is running an [amazing visualization of the proteins in the SARS-CoV-2 genome right now](https://www.nytimes.com/interactive/2020/04/03/science/coronavirus-genome-bad-news-wrapped-in-protein.html). Let's quickly take a look at the Spike protein there.

In [ ]:
# get list of genes
gene_list = [ x  for x in seq_record.features if x.type == 'CDS']
for i,x in enumerate(gene_list):
    print(x.qualifiers['gene'])

okay that S protein is the one. Let's grab it out of our sequence then

In [ ]:
# grab the spike protein
spike = gene_list[2].extract(seq_record)
#print it's sequence
print(spike.seq)

we can even turn this into a protein translation with ease

In [ ]:
print(spike.seq.translate())

note that this starts with an 'M' and ends with a stop codon '*' just like we would expect. looks awesome!

### Sequence files as dicts
If you have a large number of sequences in a file, it's usually best not to read them all in to memory at once. biopython has you covered here functions that will open up a sequence file as sa dictionary that you can query by key



In [ ]:
covid_dict = SeqIO.index("data/covid_small.fasta", "fasta")
print(list(covid_dict.keys())[0:5])

In [ ]:
seq_record = covid_dict['hCoV-19/Wuhan/WIV07/2019|EPI_ISL_402130|2019-12-30']
print(seq_record.description)
print(seq_record.seq[0:100])

### Converting between file formats

Again biopython has you covered for basic conversions between file formats. For instance

In [ ]:
from Bio import SeqIO
count = SeqIO.convert("data/covid_small.fasta", "fasta", "data/covid_small.pir", "pir")
print("Converted %i records" % count)


## Doing multiple sequence alignment
not only can biopython read and write multiple sequence alignments (MSAs) and cleanly represent them in memory, but biopython can even help you create MSAs. 
biopython can work as glue that binds together familiar bioinformatics programs, allowing us to pass information from memory to programs and then back. This depends on having "helper" programs installed that will do the heavy lifting, for instance an alignment program. When we started off this lecture we installed the aligner mafft using conda. We will use the interface to mafft to demostrate this functionality.

In particular we will do an alignment of the sequences in that SARS-CoV-2 fasta file we have been playing with

In [ ]:
import os

from Bio.Align.Applications import MafftCommandline
mafft_cline = MafftCommandline("mafft", input="data/covid_small.fasta")
#muscle_exe = "/opt/tljh/user/bin/mafft"
muscle_exe = "/Users/adk/miniconda3/bin/mafft"

assert os.path.isfile(muscle_exe), "mafft executable missing"
stdout, stderr = mafft_cline()
with open("aligned.fasta", "w") as handle:
    handle.write(stdout)
# this will take a while
# can print stderr if you want to see what happened

In [ ]:
#now suck back in the file
from Bio import AlignIO

cov_align = AlignIO.read("aligned.fasta","fasta")
print(cov_align)

these alignment objects behave as you would expect. for instance we can slice them just as we would numpy arrays to get subsets of the aligned sequences, windows of the alignment, or both

## Count number of differences
let's write a function to count the number of differences between the two sequences in our alignments. we will do this by using a simple loop and asking if the characters are different

In [ ]:
def count_diffs(align, seq1, seq2):
    diffs = 0
    for i in range(len(align[0].seq)):
        if align[seq1, i] != align[seq2, i]:
            print(f'different at position {i} -> {align[:, seq1]}')
            diffs += 1
            
    print(f'number of diffs = {diffs}')
    print(f'percentage different = {diffs / i}')

count_diffs(cov_align,0,2)

**Exercise:** rewrite the above function to only count differences between sequences that do not include alignment gaps (i.e. "-"). 

## Add reference sequence and realign
So the alignment above is great, but we don't have any of our annotations from the Genbank file on there. Let's now add the Genbank reference sequence into our alignment and realign the whole thing. The steps here will be to:
1. add the reference to the alignment object
2. write this out to a file
3. rerun the alignment step from above with the new input

In [ ]:
from Bio import Entrez
from Bio import SeqIO

# get the reference seq
# need this fake email for the service to work
Entrez.email = "fake_email@thanks.com"
with Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id="NC_045512.2") as handle:
    refseq = SeqIO.read(handle, "gb")
print("downloaded", refseq.id, seq_record.description)



In [ ]:
# then make a list from cov_align and add refseq; write to temp.fasta
longer_list = [x for x in cov_align]
longer_list.append(refseq)
SeqIO.write(longer_list, "temp.fasta", "fasta")

#now run alignment
mafft_cline = MafftCommandline("mafft", input="temp.fasta")
assert os.path.isfile(muscle_exe), "mafft executable missing"
stdout, stderr = mafft_cline()
with open("temp.aligned.fasta", "w") as handle:
    handle.write(stdout)

In [ ]:
    
#import and look at it
new_align = AlignIO.read("temp.aligned.fasta","fasta")
print(new_align)

see that last sequence name? that's our reference genome! let's see if there are any gaps inserted into the reference. this is important because it will throw off the locations of our annotations (e.g. genes) that we got with our reference sequence. To do this I'll just use the built in biopython function `count()`

In [ ]:
print(new_align[-1].seq.count("-"))

so we are actually good to go here-- things line up well to the reference. Next let's try to apply our annotation to the alignment and pull out specific regions

In [ ]:
# find S gene index
gene_list = [ x  for x in refseq.features if x.type == 'CDS']
for i,x in enumerate(gene_list):
    if x.qualifiers['gene'] == ['S']:
        print("index",i,"\n",x)


Cool! so let's get our little slice of the alignment that corresponds to this locus. There are two ways to do this. One is just to take a slice of the original alignment on the basis of the locations of features. The other is to use the SeqFeature extract function. The latter method will be much more convenient if you are trying extract genes with complex intron / exon structures

In [ ]:
from Bio.Align import MultipleSeqAlignment

seq_record_list = []
for i in range(len(new_align)):
     seq_record_list.append(gene_list[2].extract(new_align[i])) # gene_list[2] is the S gene
spike_align = MultipleSeqAlignment(seq_record_list)

# could also do this with a list comprehension
#spike_align = MultipleSeqAlignment([gene_list[2].extract(new_align[i]) for i in range(len(new_align))])

print(spike_align)

In [ ]:
# first way-- take a slice
spike_align = new_align[:,gene_list[2].location.start.position:gene_list[2].location.end.position]



okay great so we've go the subsection of the alignment that pertains to the Spike locus. now let's do something with it! In particular let's compute the number of nucleotide differences among Spike sequences in our sample, let's build a matrix of them, and let's visualize them. Here is our simple function

In [ ]:
def count_diffs2(align, seq1, seq2):
    diffs = 0
    for i in range(len(align[0].seq)):
        if align[seq1, i] != align[seq2, i] and align[seq1, i] != '-' \
            and align[seq2, i] != "-":
            diffs += 1
    return(diffs)

now let's visualize these as a heatmap. I'll use the `imshow` function as a quick and easy way to do this sort of visualization

In [ ]:
from matplotlib import pyplot as plt

n = len(spike_align)
dist_mat = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        dist_mat[i,j] = count_diffs2(spike_align,i,j)

plt.imshow(dist_mat, cmap='hot', interpolation='nearest')
plt.xticks(range(n), [x.id for x in spike_align])
plt.yticks(range(n), [x.id for x in spike_align])
plt.xticks(rotation=90)


So here we are visualizing the number of nucleotide differences in pairwise comparisons among genomes. A question then-- why is this figure symmetric about the diagonal? 

We can also make a histogram to look at the frequency distribution of the number of nucleotide differences between species

In [ ]:
plt.hist(dist_mat.flatten(), bins=20)

## Digging deeper into the past-- BLAST
So now that we've got our alignment of the Spike locus a bit cleaned up, let's see if we can do a quick comparison using BLAST and see what we can pick up. BLAST (basic local alignment search tool) is way to search amino-acid, DNA and/or RNA sequences. A BLAST search enables a researcher to compare a subject protein or nucleotide sequence (called a query) with a library or database of sequences, and identify library sequences that resemble the query sequence above a certain threshold. The web interface to BLAST is [here](https://blast.ncbi.nlm.nih.gov/Blast.cgi)

Let's try pulling out the amino acid sequence of SARS-CoV-2 Spike and BLASTing that against the NCBI database. To do that let's pull the sequence feature and translate it


In [ ]:
ref_spike_prot = spike_align[-1].seq.translate()
print(ref_spike_prot)

now let's copy and paste that into the BLAST window... and hit go. We will use the protein BLAST option (blastp). This may take a few minutes depending on traffic to the server. Looking over my results here I get a bunch of high quality hits to SARS-CoV-2 sequences (makes sense) and then one hit to a different sequence

<img src=imgs/blast.png>

a bat coronoavirus RaTG13!!! 

so let's do what comes naturally-- let's download that sequence, and align it to what we have right now. In fact this would be a great homework problem for you... to get you started let's use that id to download that sequence using it's unique identifier

In [ ]:
Entrez.email = "fake_email@thanks.com"
with Entrez.efetch(db="protein", rettype="gb", retmode="text", id="QHR63300.2") as handle:
    bat_spike = SeqIO.read(handle, "gb")
print("downloaded", refseq.id, seq_record.description)



In [ ]:
print(bat_spike)

awesome! you'll note that i've downloaded the protein sequence here and not the nucleotide sequence of the gene nor the genome. We will play with this further in the homework